In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [19]:
train_df = pd.read_csv('./sub_train.csv')
test_df = pd.read_csv('./sub_test.csv')

In [20]:
target=["contest-tmp2m-14d__tmp2m"]
selected_cols = [c for c in train_df.columns if c not in [target[0], 'startdate']]

In [21]:
pseudo_df = pd.read_csv('./pseu.csv')
add_data_df = test_df[selected_cols]
add_data_df[target[0]] = pseudo_df[target[0]]

/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
new_train_df = pd.concat([train_df[selected_cols + target], add_data_df[selected_cols + target]], axis=0)

In [23]:
Xtest = add_data_df[selected_cols].values

In [24]:
del train_df, test_df, add_data_df

In [25]:
X = new_train_df[selected_cols].values
y = new_train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, learning_rate=0.12, task_type="GPU",
                           devices='1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=1000, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)


5it [00:00,  7.79it/s]


(407088, 59) (407088,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 4363.5625 Total: 24268.3125


0:	learn: 8.6746706	test: 8.6778631	best: 8.6778631 (0)	total: 51.9ms	remaining: 4m 19s
1000:	learn: 0.5463133	test: 0.5614403	best: 0.5614403 (1000)	total: 18.4s	remaining: 1m 13s
2000:	learn: 0.4107029	test: 0.4326649	best: 0.4326649 (2000)	total: 22s	remaining: 33s
3000:	learn: 0.3434990	test: 0.3698492	best: 0.3698492 (3000)	total: 25.7s	remaining: 17.1s
4000:	learn: 0.3008216	test: 0.3306348	best: 0.3306348 (4000)	total: 29.4s	remaining: 7.33s
4999:	learn: 0.2705425	test: 0.3029381	best: 0.3029381 (4999)	total: 33.2s	remaining: 0us
bestTest = 0.3029381039
bestIteration = 4999


 20%|█████████████████▌                                                                      | 1/5 [00:39<02:39, 39.79s/it]

...

 60%|████████████████████████████████████████████████████▊                                   | 3/5 [01:30<00:56, 28.42s/it]

0:	learn: 8.6633532	test: 8.6827796	best: 8.6827796 (0)	total: 4.64ms	remaining: 23.2s
1000:	learn: 0.5431288	test: 0.5635387	best: 0.5635387 (1000)	total: 3.69s	remaining: 14.7s
2000:	learn: 0.4089897	test: 0.4351142	best: 0.4351142 (2000)	total: 7.29s	remaining: 10.9s
3000:	learn: 0.3430221	test: 0.3730782	best: 0.3730782 (3000)	total: 11.3s	remaining: 7.55s
4000:	learn: 0.3003530	test: 0.3332594	best: 0.3332594 (4000)	total: 15.2s	remaining: 3.79s
4999:	learn: 0.2706952	test: 0.3058717	best: 0.3058717 (4999)	total: 18.8s	remaining: 0us
bestTest = 0.3058717035
bestIteration = 4999


 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [01:55<00:27, 27.19s/it]

0:	learn: 8.6738417	test: 8.6387045	best: 8.6387045 (0)	total: 4.96ms	remaining: 24.8s
1000:	learn: 0.5444336	test: 0.5596470	best: 0.5596470 (1000)	total: 3.62s	remaining: 14.5s
2000:	learn: 0.4085173	test: 0.4296259	best: 0.4296259 (2000)	total: 7.53s	remaining: 11.3s
3000:	learn: 0.3408190	test: 0.3659387	best: 0.3659387 (3000)	total: 11.2s	remaining: 7.43s
4000:	learn: 0.2989077	test: 0.3275918	best: 0.3275918 (4000)	total: 14.9s	remaining: 3.71s
4999:	learn: 0.2692991	test: 0.3008209	best: 0.3008209 (4999)	total: 18.7s	remaining: 0us
bestTest = 0.3008209008
bestIteration = 4999


100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:20<00:00, 28.15s/it]


In [26]:
preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(Xtest))
s = 0
for pred in preds:
    s += pred
s = s/5
result_df = pd.read_csv('./sample_solution.csv')

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
result_df['contest-tmp2m-14d__tmp2m'] = s
result_df.to_csv('submission_pseudo_labeling_v1.csv', index=False)

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 40.72it/s]


cos_sim with best submission: 0.9998425955358088
